# Титаник

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [59]:
data = pd.read_csv('titanic.csv')

In [20]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# ЧТО ДЕЛАТЬ ДАЛЬШЕ ?!








In [21]:
data = data[['Survived','Pclass', 'Fare','Age','Sex']]

In [22]:
data.head()

,Survived,Pclass,Fare,Age,Sex
0,0,3,7.2500,22.0,male
1,1,1,71.2833,38.0,female
2,1,3,7.9250,26.0,female
3,1,1,53.1000,35.0,female
4,0,3,8.0500,35.0,male


In [23]:
data['Sex'] = data['Sex'] == 'female'
data['Sex'] = data['Sex'].astype(int)

In [11]:
# выбор значений где нет NaN
data = data[np.isnan(data['Age']) == False]
data = data[np.isnan(data['Pclass']) == False]
data = data[np.isnan(data['Fare']) == False]
data = data[np.isnan(data['Sex']) == False]

In [24]:
# замена NaN на какое-то значение
data = data.fillna(-1)
# и можно подсчитать количество пропусков (но это надо было делать заранее)
data['nan'] = data.isnull().sum(axis=1)

In [25]:
data.head()

,Survived,Pclass,Fare,Age,Sex,nan
0,0,3,7.2500,22.0,0,0
1,1,1,71.2833,38.0,1,0
2,1,3,7.9250,26.0,1,0
3,1,1,53.1000,35.0,1,0
4,0,3,8.0500,35.0,0,0


In [26]:
X = data.drop('Survived', 1)
y = data['Survived']
clf = DecisionTreeClassifier(random_state = 241) # random state чтобы запомнить состояние
clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=241, splitter='best')

In [27]:
importances = clf.feature_importances_
print importances

[ 0.10634357  0.32317404  0.25930655  0.31117584  0.        ]


## Кросс-Валидация

Удобно разбивать по фолдам и проверить сетку параметров

In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import grid_search

method = DecisionTreeClassifier()

parameters = {"criterion": ["gini", "entropy"],
              "min_samples_split": [2, 10, 20,50],
              "max_depth": [None, 2, 5, 10,30],
              "min_samples_leaf": [1, 5, 10],
              "max_leaf_nodes": [None, 5, 10, 20,50],
              }

clf = grid_search.GridSearchCV(method, parameters,cv=5,scoring='accuracy')
clf.fit(X, y)
print "Best score={} with params= {} for method {}".format(clf.best_score_,clf.best_params_, method)

best_est = clf.best_estimator_
#best_est.fit(X_train,y_train)

#tp = best_est.predict(X_test)
#sc = accuracy_score(tp,y_test)

Best score=0.820426487093 with params= {'min_samples_split': 2, 'max_leaf_nodes': None, 'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 10} for method DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')


## Работа с нечисленными признаками

In [40]:
# one_hot encoding
one_hot = pd.get_dummies(data['Embarked'])

In [52]:
# label encoder
label_encoder = pd.factorize(data['Embarked'])[0]

## Текст

In [66]:
data['Name'] = data['Name'].fillna('nan')

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf_vec = TfidfVectorizer()
count_vec = CountVectorizer()
tfidf_x = tfidf_vec.fit_transform(data['Name'])
count_x = count_vec.fit_transform(data['Name'])

In [74]:
#tfidf_x - разреженная матрица
tfidf_x.toarray() # np массив

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])